# Lab Guide 02 - Aprendizagem de Máquina
Nesse experimento, iremos trabalhar com o problema de regressão sobre o dataset clássico *The Boston Housing Dataset*, conhecido como Boston house-price. Veja mais informações [aqui](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).

Ao completar essa tarefa, você:
* Conhecerá, ainda que superficialmente, diversas bibliotecas Python para Ciência de Dados e Aprendizagem de Máquina;
* Carregará um conjunto de dados para problema de regressão;
* Terá noções básicas de visualização e normalização de conjuntos de dados;
* Treinará diversos modelos de regressão sobre esse dataset; e
* Visualizará o resultado produzido pelo seu modelo.

**NOTA:** esse Lab Guide é baseado no material da Huawei, sendo disponibilizado para uso exclusivo pelos alunos do curso e não deve ser divulgado sem autorização prévia e expressa.




## [Passo 01] Carregando as Dependências

Nesse passo iremos carregar diversas bibliotecas para auxiliar em Tarefas de aprendizagem de Máquina.

In [ ]:
# Filtragem dos avisos, pois se trata de um notebook onde o foco é resultado.
# Aplicações devem evitar isso para não perder evidências úteis de erros.
import warnings
warnings.filterwarnings('ignore')

# suporte a arrays e matrizes multidimensionais
import numpy as np
# plotagem semelhante ao MATLAB
import matplotlib as mpl
# diversas funções de plotagem
import matplotlib.pyplot as plt 
# manipulação e análise de dados, tais como tabelas e séries temporais
import pandas as pd
# distribuições de probabilidade e funções estatísticas
import scipy.stats as st
# visualização de dados em alto nível
import seaborn as sns

# padronização de características: média zero e variância unitária
from sklearn.preprocessing import StandardScaler
# divisão de arrays e matrizes em subconjuntos, útil para cross validation
from sklearn.model_selection import train_test_split
# busca em gradem, útil para seleção de hiperparâmetros
from sklearn.model_selection import GridSearchCV
# coeficiente de determinação para regressão
from sklearn.metrics import r2_score

# carregamento do dataset Boston House Prices
from sklearn.datasets import load_boston
# modelos para regressãp
from sklearn.linear_model import RidgeCV, LassoCV, LinearRegression, ElasticNet
# SVM para problemas de regressão
from sklearn.svm import SVR
# algoritmos ensemble (comitê de modelos) para regressão
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# algoritmo Extreme Gradient Boosting, do tipo ensemble
from xgboost import XGBRegressor

## [Passo 02] Carregando e Visualizando o Dataset

A função utilitária [sklearn.datasets.load_boston()](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html?highlight=load_boston#sklearn.datasets.load_boston) carrega o dataset.



In [ ]:
boston = load_boston()
#x features, and y labels.
x = boston.data
y = boston.target

#Display related attributes.
print('Feature column name')
print(boston.feature_names)
print('Sample data volume: %d, number of features: %d'% x.shape)
print('Target sample data volume: %d'% y.shape[0])

Converte o dataset para um objeto *DataFrame* da biblioteca Pandas, o que facilita sua visualização e tratamento dos dados.

In [ ]:
x = pd.DataFrame(boston.data, columns=boston.feature_names)
x.head()

Visualização da distribuição dos rótulos *sem* estimativa de densidade por kernel e curva normal para ajuste.

In [ ]:
plt.figure()
sns.distplot( tuple(y), kde=False, fit=st.norm)

Agora iremos particionar as amostras do dataset, de modo que sejam usados dados distintos para treinamento e para teste. Isso permite avaliar o quanto o modelo de regressão é capaz de generalizar a partir dos dados observados no treinamento.

Note que *random_state* é usado para definir um particionamento previsível para fins da atividade neste notebook.

In [ ]:
# 80% das amostras são usadas para treinamento, 20% para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=28)

# padronização dos vetores característicos (features)
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

# visualização da subdivisão
plt.figure()
sns.distplot( tuple(y_train), kde=False, fit=st.norm, label='Train')
plt.figure()
sns.distplot( tuple(y_test), kde=False, fit=st.norm, label='Test')

## [Passo 03] Usando Técnicas de Regressão
Uma vez que os dados estejam separados, podemos lançar mão de diversos métodos de Aprendizagem de Máquina. A interface padronizada do pacote **sklearn** é bastante útil para esse propósito.


In [ ]:
# nomes identificando os modelos de regressão
names = ['Linear',
         'Ridge',
         'Lasso',
         'Random Forrest',
         'GBDT',
         'Support Vector Regression',
         'ElasticNet',
         'XgBoost'
        ]

# instancia diversos métodos de regressão correspondentes aos nomes
models = [LinearRegression(),
          RidgeCV(alphas=(0.001,0.1,1),cv=3),
          LassoCV(alphas=(0.001,0.1,1),cv=5),
          RandomForestRegressor(n_estimators=10),
          GradientBoostingRegressor(n_estimators=30),
          SVR(),
          ElasticNet(alpha=0.001,max_iter=10000),
          XGBRegressor()]

def R2(model, x_train, x_test, y_train, y_test):
  """ Ajusta o modelo aos dados de treinamento (x_train e y_train).
      Computa o Coeficiente de Determinação para a predição realizada
      por aquele modelo para os dados de teste: uma predição perfeita
      possui score 1.0
  """
  model_fitted = model.fit(x_train,y_train)
  y_pred = model_fitted.predict(x_test)
  score = r2_score(y_test, y_pred)
  return score

# efetua o ajuste dos modelos e exibe o resultado.
for name,model in zip(names,models):
  score = R2(model,x_train, x_test, y_train, y_test)
  print('{}: {:.6f}, {:.4f}'.format(name,score.mean(),score.std()))

## [Passo 04] Otimização de Hiperparâmetros
Nessa etapa, procederemos com uma busca em grade para realizar a otimização dos hiperparâmetros. O método de regressão escolhido é o SVR, assim os seguintes parâmetros formarão a grade de busca:
* Tipo de kernel, *linear* ou *RBF*;
* C, o peso da penalidade de erro; e
* Gamma, o parâmetro do kernel RBF.

Tenha paciência ao executar a célula seguinte. Como se trata de uma busca exaustiva com validação cruzada (**3-fold cross validation**), é normal que o resultado demore um pouco para ser produzido. 

In [ ]:
# define os parâmetros da busca exaustiva: todas as combinações
parameters = {
    'kernel': ['linear', 'rbf' ],
        'C' : [0.1  , 0.5 ,0.9,1,5],
    'gamma' : [0.001, 0.01,0.1,1]
}

# busca exaustiva (grid search) com 3-fold cross validation
print("starting exhaustive search... (please wait)")
model = GridSearchCV( SVR(), param_grid=parameters, cv=3 )
model.fit(x_train, y_train)

Agora podemos visualizar os hiperparâmetros que produziram os melhores resultados. Compare o valor do novo score obtido após a otimização com o obtido anteriormente: um salto de aproximadamente *0,517* para *0,618* .

In [ ]:
print('Optimal parameter list:', model.best_params_)
print('Optimal model:', model.best_estimator_)
print('Optimal R2 value (train):', model.best_score_)

# verifica o score nos dados de teste
y_pred = model.predict(x_test)
best_score_test = r2_score(y_test, y_pred)
print('Optimal R2 value (test) :', best_score_test )

Finalmente, podemos plotar um gráfico para comparar a previsão produzida pelo modelo com o resultado ideal, já conhecido nos dados de teste.

In [ ]:
ln_x_test = range(len(x_test))
y_predict = model.predict(x_test)

plt.figure(figsize=(16,8), facecolor='w')
# valores originais, conhecidos
plt.plot (ln_x_test, y_test, 'r-', lw=2, label=u'Value')
# valores previstos pelo modelo após a busca por parâmetros
plt.plot (ln_x_test, y_predict, 'g-', lw = 3, label=u'Estimated value of the SVR algorithm, $R^2$=%.3f' %
(best_score_test))

# plotagem
plt.legend(loc ='upper left')
plt.grid(True)
plt.title('Boston Housing Price Forecast (SVR)')
plt.xlim(0, 101)
plt.show()